# Detectron2 Detect Chess Detection

<img src="chest.JPG" />

In this exercise, we will see how to finetune a model to adapt it to a specific task. 

We are going to use a pretrain model for a new task: Chess detection.

> ⚠️⚠️ **Important**: Run this notebook on Google Colab, with GPU enabled.

> ⚠️ In this exercise, we gave you some pieces of code that you need to complete : look for the `# TODO`

## Introduction to fine-tuning a model 

## Get data

In [ ]:
!pip install pyyaml==5.1

# install detectron from source
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

# torch
import torch
print(torch.__version__, torch.cuda.is_available())

# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt

# import some common libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

Before we can start training our model we need to download our data-set. In this case we will use my microcontroller detection data-set, which is [available on Kaggle](https://www.kaggle.com/tannergi/microcontroller-detection).

In [2]:
# Install Kaggle API
!pip install -q kaggle
!pip install -q kaggle-cli

In order to use the Kaggle’s public API, you must first authenticate using an API token. From the site header, click on your user profile picture, then on “My Account” from the dropdown menu. This will take you to your account settings at https://www.kaggle.com/account. Scroll down to the section of the page labelled API:

To create a new token, click on the “Create New API Token” button. This will download a fresh authentication token onto your machine. Copy your username and the key generated in the kaggle.json file and use it to complete the code snippet given below.

In [3]:
import os
os.environ['KAGGLE_USERNAME'] = "<username>" 
os.environ['KAGGLE_KEY'] = "<password>"

In [ ]:
!kaggle datasets download -d tannergi/chess-piece-detection

In [ ]:
!unzip chess-piece-detection.zip

## Register data-set

In order to use a dataset with Detectron2 we need to register it. For more information check out the [official documentation](https://detectron2.readthedocs.io/tutorials/datasets.html#register-a-dataset).

In [6]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode
import itertools
import cv2
import xml.etree.ElementTree as ET


# we give you a function that loads the dataset into detectron2's standard format
def get_dataset_dicts(annotation_path, image_path):
    classes = ['white-rook', 'white-knight', 'white-bishop', 'white-king', 'white-queen', 'white-pawn', 'black-rook', 'black-knight', 'black-bishop', 'black-king', 'black-queen', 'black-pawn']

    dataset_dicts = []
    for filename in os.listdir(annotation_path):
        record = {}
        root = ET.parse(os.path.join(annotation_path, filename)).getroot()
        record["file_name"] = os.path.join(image_path, root.find('filename').text)
        record["height"] = 2248
        record["width"] = 4000

        objs = []
        for member in root.findall('object'):
          obj = {
              'bbox': [int(member[4][0].text), int(member[4][1].text), int(member[4][2].text), int(member[4][3].text)],
              'bbox_mode': BoxMode.XYXY_ABS,
              'category_id': classes.index(member[0].text),
              "iscrowd": 0
          }
          objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [7]:
from detectron2.data import DatasetCatalog, MetadataCatalog

classes = ['white-rook', 'white-knight', 'white-bishop', 'white-king', 'white-queen', 'white-pawn', 'black-rook', 'black-knight', 'black-bishop', 'black-king', 'black-queen', 'black-pawn']

DatasetCatalog.register('chess_piece_dataset', lambda: get_dataset_dicts('Chess Detection/annotations', 'Chess Detection/images'))
MetadataCatalog.get('chess_piece_dataset').set(thing_classes=classes)
chess_metadata = MetadataCatalog.get('chess_piece_dataset')

We can check if our code works correctly by plotting a few images with visualizer

# Q1 . Display few images 

We can use :  
    - dataset_dicts = DatasetCatalog.get('chess_piece_dataset')  
in order to get the dataset

In [ ]:
import random
from detectron2.utils.visualizer import Visualizer

dataset_dicts = DatasetCatalog.get('chess_piece_dataset')
for d in random.sample(dataset_dicts, 10):
    # TODO : write code to display images

# Q2 . Train model

### load configuration

In order to fine tune modal you have to : 
- select a pretraining model 
- define dataset.train and dataset.test
- define number of classes

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(# TODO )
cfg.DATASETS.TRAIN = ('chess_piece_dataset',)
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = # TODO
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = # TODO

### Then load the model (same a DefaultPredictor but you have to use DefaultTrainer)

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = # TODO

In [ ]:
### TODO : finally, run the training. 

# Q3. Use model for inference

Now, we can perform inference on our validation set by creating a predictor object.

The model is saved if the cfg.OUTPUT_DIR path

### Q3.1 : print cfg.OUTPUT_DIR

In [ ]:
# TODO : print cfg.OUTPUT_DIR

### Q3.2 : change cfg.MODEL.WEIGHTS with the path of the model save

In [ ]:
cfg.MODEL.WEIGHTS = # TODO : path to model
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("chess_piece_dataset", )
predictor = DefaultPredictor(cfg)

### Q3.3 : display prediction

In [ ]:
from detectron2.utils.visualizer import ColorMode
import random

dataset_dicts = DatasetCatalog.get('chess_piece_dataset')
for d in random.sample(dataset_dicts, 5):    
    # TODO : get image, predict and display prediction